Transfer Learning Xception

In [ ]:
import tensorflow as tf
import os
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

if not os.path.isdir('models'):
    os.mkdir('models')
    
print('TensorFlow version:', tf.__version__)
print('Is using GPU?', tf.test.is_gpu_available())

TensorFlow version: 2.3.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is using GPU? True


In [ ]:
import cv2
from glob import glob
from scipy import stats as s
import keras
from keras.models import Sequential
import tensorflow.keras.applications.xception
from keras.applications.xception import Xception
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('Data/train_new.csv')
train.head()

,image,class
0,Harassment_h36.mp4_frame34.jpg,1
1,Normal_n27.mp4_frame20.jpg,0
2,Normal_n46.mp4_frame43.jpg,0
3,Harassment_h3.mp4_frame6.jpg,1
4,Harassment_h15.mp4_frame10.jpg,1


In [ ]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('train/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255.0
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|██████████| 4227/4227 [01:47<00:00, 39.30it/s]


(4227, 224, 224, 3)

In [ ]:
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [ ]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [ ]:
# creating the base model of pre-trained Xception model
base_model = Xception(weights='imagenet', include_top=False)

83689472/83683744 [==============================] - 1s 0us/step


In [ ]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(3381, 7, 7, 2048)

In [ ]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(846, 7, 7, 2048)

In [ ]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(3381, 7*7*2048)
X_test = X_test.reshape(846, 7*7*2048)

In [ ]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max
X_train.shape

(3381, 100352)

In [ ]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(100352,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              102761472 
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               3

In [ ]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/200
27/27 [==============================] - 5s 197ms/step - loss: 0.9191 - accuracy: 0.6025 - val_loss: 0.3682 - val_accuracy: 0.8558
Epoch 2/200
27/27 [==============================] - 25s 930ms/step - loss: 0.3921 - accuracy: 0.8231 - val_loss: 0.1302 - val_accuracy: 0.9657
Epoch 3/200
27/27 [==============================] - 33s 1s/step - loss: 0.1406 - accuracy: 0.9479 - val_loss: 0.0382 - val_accuracy: 0.9882
Epoch 4/200
27/27 [==============================] - 29s 1s/step - loss: 0.0533 - accuracy: 0.9811 - val_loss: 0.0286 - val_accuracy: 0.9870
Epoch 5/200
27/27 [==============================] - 30s 1s/step - loss: 0.0340 - accuracy: 0.9882 - val_loss: 0.0201 - val_accuracy: 0.9917
Epoch 6/200
27/27 [==============================] - 32s 1s/step - loss: 0.0323 - accuracy: 0.9896 - val_loss: 0.0087 - val_accuracy: 0.9965
Epoch 7/200
27/27 [==============================] - 32s 1s/step - loss: 0.0249 - accuracy: 0.9926 - val_loss: 0.0073 - val_accuracy: 0.9976
Epoch 8/

In [ ]:
# getting the test list
f = open("test.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating the dataframe
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.head()

,video_name
0,Harassment/Harassment_h7.mp4
1,Harassment/Harassment_h11.mp4
2,Harassment/Harassment_h16.mp4
3,Harassment/Harassment_h20.mp4
4,Harassment/Harassment_h23.mp4


In [ ]:
# creating the tags
train = pd.read_csv('Data/train_new.csv')
y = train['class']
y = pd.get_dummies(y)

In [ ]:

from scipy import stats as s
# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    cap = cv2.VideoCapture('Data/'+videoFile)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    # removing all other files from the temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % 3 == 0):
            # storing the frames of this particular video in temp folder
            filename ='temp/' + "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    
    # reading all the frames from temp folder
    images = glob("temp/*.jpg")
    
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255.0
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*2048)
    # predicting tags for each array
    prediction = model.predict_classes(prediction_images)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    if (videoFile.split('/')[1].split('_')[0] == 'Harassment'):
        actual.append(1)
    else:
        actual.append(0)
    # actual.append(videoFile.split('/')[1].split('_')[1])

  0%|          | 0/20 [00:00<?, ?it/s]

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


100%|██████████| 20/20 [01:27<00:00,  4.37s/it]


In [ ]:
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

95.0